In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table,con,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursorUAT.execute(sql_query)
            connUAT.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            con.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

sql_check = '''
            select * from staging.stg_tmdb_json_movie_metadata
            '''
sql_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
sql_json.sort_values(by='id',inplace=True)
sql_json

,id,request_json
86196,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
86160,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
86161,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
86162,6,"{""adult"": false, ""backdrop_path"": ""/iUUpKunmBN..."
86163,11,"{""adult"": false, ""backdrop_path"": ""/2w4xG178Rp..."
...,...,...
70144,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
86156,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
86157,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
86158,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


In [6]:
def check_list(sql_table_object,connSQL):
    sql_query = f'''
                select distinct id from {sql_table_object}
                '''
    list_sql_object = selectdf(sql_query,connSQL)
    list_sql_object.sort_values(by='id',inplace=True)
    
    sql_query = '''
            select distinct tmdbid id from staging.stg_links
            where tmdbid is not null
            order by tmdbid
            '''
    list_sql_all = selectdf(sql_query,connSQL)
    list_sql_all.sort_values(by='id',inplace=True)
    
    list_crwal = list(set(list_sql_all['id']) - set(list_sql_object['id']))
    list_crwal.sort()
    return list_crwal

list_m = check_list('staging.stg_tmdb_json_movie_credits',connUAT)
list_m


[2,
 3,
 5,
 6,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 35,
 38,
 55,
 58,
 59,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 127,
 128,
 129,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 185,
 186,
 187,
 189,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 

In [5]:
    connSQL = connUAT
    sql_query = '''
            select distinct tmdbid id from staging.stg_links
            where tmdbid is not null
            order by tmdbid
            '''
    list_sql_all = selectdf(sql_query,connSQL)
    list_sql_all.sort_values(by='id',inplace=True)
    list_sql_all

,id
0,2
1,3
2,5
3,6
4,11
...,...
86369,1144338
86370,1145358
86371,1146246
86372,1146249


In [12]:
# fall_m = sql_json[sql_json['request_json']=='{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}']

### sử dụng:
- belongs_to_collection: dùng id để lọc những film liên quan lẫn nhau
- Dữ liệu có 126 movieid ko có mã tbdmid
- 1071 movieid có mã tbdmid những ko có dữ liệu trên server 
    - code: tbdm movies_info[movies_info['tmdbId'].isin(list(fall_m['id']))]


In [13]:
# tmdb_data = json.loads(sql_json[sql_json['id']==11]['request_json'].values[0])
# # tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [14]:
tmdb_data = json.loads(sql_json[sql_json['id']==1891]['request_json'].values[0])
# tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [15]:
# [tmdb_data['production_countries'][0]['name'],tmdb_data['spoken_languages'][0]['english_name']
#  ,tmdb_data['production_companies'][0]['name']
#  ,tmdb_data['overview'],tmdb_data['status'],tmdb_data['release_date']
# ]

In [16]:
# movies_links[movies_links['tmdbId'].isnull()]

In [17]:
movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links.sort_values(by='tmdbId',inplace=True)
movies_info = links.merge(movies, on='movieId', how='left')
del movies_info['genres']
# movies.sort_values(by='tmdbId',inplace=True)
movies_info


,movieId,imdbId,tmdbId,title
0,4470,94675,2.0,Ariel (1988)
1,61724,92149,3.0,Shadows in Paradise (Varjoja paratiisissa) (1986)
2,18,113101,5.0,Four Rooms (1995)
3,479,107286,6.0,Judgment Night (1993)
4,260,76759,11.0,Star Wars: Episode IV - A New Hope (1977)
...,...,...,...,...
86532,222211,7688638,NaN,Good Girls Get High (2018)
86533,224398,5868094,NaN,She Loves Me (2016)
86534,234301,5714216,NaN,Black Butler: Book of Murder (2014)
86535,237231,9372822,NaN,Tenderness (2018)


In [38]:
movies_info[movies_info['movieId']==260]

,movieId,imdbId,tmdbId,title
4,260,76759,11.0,Star Wars: Episode IV - A New Hope (1977)


In [18]:
movies_info.isnull().sum()

movieId      0
imdbId       0
tmdbId     126
title        0
dtype: int64

In [19]:
# SQL_push(
#          sql_table_name='staging.stg_grouplen_movieinfo',
#          python_table=movies_info,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [20]:
# DONE
movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
movies1 = movies[['movieId','genres']]
max_col = [len(i) for i in list(movies1['genres'].str.split('|'))]
movies_genres = pd.DataFrame()

for i in range(max(max_col)):
    data = movies.copy()
    data['genres'] = data['genres'].str.split('|',expand=True)[i]
    data = data[data['genres'].isnull()==False]
    movies_genres = pd.concat([movies_genres,data])

movies_genres.sort_values(by='movieId',ascending=True,inplace=True)
del movies_genres['title']
movies_genres

,movieId,genres
0,1,Adventure
0,1,Fantasy
0,1,Animation
0,1,Comedy
0,1,Children
...,...,...
86533,288971,Horror
86534,288975,Documentary
86535,288977,Thriller
86535,288977,Crime


In [21]:
movies_genres.isnull().sum()

movieId    0
genres     0
dtype: int64

In [22]:
# SQL_push(
#          sql_table_name='staging.stg_grouplen_moviesgenres',
#          python_table=movies_genres,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [23]:
# DONE
moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags['date_time'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
# moives_tags['date'] = pd.to_datetime(moives_tags['date_time'],format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
del moives_tags['timestamp']
# tags.sort_values(by='tmdbId',inplace=True)
moives_tags

,userId,movieId,tag,date_time
0,10,260,good vs evil,2015-05-03 15:22:38
1,10,260,harrison ford,2015-05-03 15:21:45
2,10,260,sci-fi,2015-05-03 15:22:18
3,14,1221,al pacino,2011-07-25 13:32:36
4,14,1221,mafia,2011-07-25 13:32:26
...,...,...,...,...
2328310,330923,176599,politically correct,2017-10-09 11:11:31
2328311,330933,3317,coming of age,2012-10-26 19:23:04
2328312,330933,3317,sexuality,2012-10-26 19:23:09
2328313,330947,5782,not luc besson,2006-07-28 18:21:42


In [ ]:
# moives_tags_agg = moives_tags[['movieId','tag']].drop_duplicates()
# moives_tags_agg = moives_tags_agg.groupby('movieId').agg(keywords=('tag',list)).reset_index()
# moives_tags_agg['keywords'] = moives_tags_agg['keywords'].astype('str')
# moives_tags_agg

,movieId,keywords
0,1,"['animation', 'friendship', 'toys', 'disney', ..."
1,2,"['animals', 'based on a book', 'fantasy', 'mag..."
2,3,"['sequel', 'moldy', 'old', 'old age', 'old men..."
3,4,"['characters', 'chick flick', 'girl movie', 'r..."
4,5,"['family', 'pregnancy', 'wedding', '4th wall',..."
...,...,...
53447,288765,"['post-apocalyptic', 'survival', 'tw suicide',..."
53448,288779,['don camillo series']
53449,288849,"['addiction', 'animation', 'short film']"
53450,288937,['anime']


In [54]:
# SQL_push(
#          sql_table_name='staging.stg_keywords_list',
#          python_table=moives_tags_agg,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [40]:
test = moives_tags[moives_tags['movieId']==260][['movieId','tag']]
test.drop_duplicates(inplace=True)
test[test['tag'].str.contains('jedi')]

,movieId,tag
21775,260,jedi
136929,260,a boy grows into a jedi
525395,260,i wanna be a jedi
1060916,260,jedi master
1060917,260,jedi mind trick
1060918,260,jedi training
2310866,260,jedi wars


In [ ]:
moives_tags.isnull().sum()

userId        0
movieId       0
tag          27
date_time     0
dtype: int64

In [25]:
# SQL_push(
#             sql_table_name= 'staging.stg_grouplen_moviestags',
#             python_table= moives_tags,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [26]:
# DONE
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['date_time'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings

,userId,movieId,rating,date_time
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39
...,...,...,...,...
33832157,330975,8340,2.0,2004-08-04 01:34:16
33832158,330975,8493,2.5,2004-08-04 02:15:09
33832159,330975,8622,4.0,2004-08-04 01:09:37
33832160,330975,8665,3.0,2004-08-04 01:09:25


In [27]:
movies_ratings.isnull().sum()

userId       0
movieId      0
rating       0
date_time    0
dtype: int64

In [28]:
# SQL_push(
#             sql_table_name= 'staging.stg_grouplen_moviesratings',
#             python_table= movies_ratings,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [29]:
# DONE
genome_s = pd.read_csv(os.path.join(pathfolder,'genome-scores.csv'))
genome_s

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300
...,...,...,...
18472123,288167,1124,0.09875
18472124,288167,1125,0.02950
18472125,288167,1126,0.02275
18472126,288167,1127,0.11225


In [30]:
genome_s.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

In [31]:
# SQL_push(
#             sql_table_name= 'staging.stg_genome_scores',
#             python_table= genome_s,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [32]:
# DONE
genome_t = pd.read_csv(os.path.join(pathfolder,'genome-tags.csv'))
genome_t

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [33]:
genome_t.isnull().sum()

tagId    0
tag      0
dtype: int64

In [34]:
# SQL_push(
#             sql_table_name= 'staging.stg_genome_tags',
#             python_table= genome_t,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [55]:
connUAT.commit()
connUAT.close()